In [2]:
import pandas as pd
import requests
import json
from tqdm import tqdm

In [3]:
%load_ext jupyter_black

## Determine join property & fetch all eligable external ids

In [ ]:
joinProp = input("What property are you going to join the tables on?")
with open("colNamePid.json", "w+") as file:
    config = json.load(file)
    config["__joinProp"] = joinProp
    json.dump(config, file)

## Buisiness logic for retrieving the external db

In [6]:
pd.set_option("display.max_columns", None)

In [7]:
current = pd.read_csv(
    "https://theunitedstates.io/congress-legislators/legislators-current.csv"
)
historic = pd.read_csv(
    "https://theunitedstates.io/congress-legislators/legislators-historical.csv"
)
politicians = pd.concat([current, historic])
politicians.reset_index(
    drop=True, inplace=True
)  # Reset duplicate indexes after merging
politicians.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,NaN,1.0,Democrat,https://www.brown.senate.gov,503 Hart Senate Office Building Washington DC ...,202-224-2315,https://www.brown.senate.gov/contact/,http://www.brown.senate.gov/rss/feeds/?type=al...,SenSherrodBrown,43910797.0,SenatorSherrodBrown,SherrodBrownOhio,UCgy8jfERh-t_ixkKKoCmglQ,NaN,B000944,136.0,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,NaN,1.0,Democrat,https://www.cantwell.senate.gov,511 Hart Senate Office Building Washington DC ...,202-224-3441,https://www.cantwell.senate.gov/public/index.c...,http://www.cantwell.senate.gov/public/index.cf...,SenatorCantwell,117501995.0,senatorcantwell,SenatorCantwell,UCN52UDqKgvHRk39ncySrIMw,NaN,C000127,172.0,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,NaN,1.0,Democrat,https://www.cardin.senate.gov,509 Hart Senate Office Building Washington DC ...,202-224-4524,https://www.cardin.senate.gov/contact/,http://www.cardin.senate.gov/rss/feeds/?type=all,SenatorCardin,109071031.0,senatorbencardin,senatorcardin,UCiQaJnMzlfzzG3VESgyZChA,NaN,C000141,174.0,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,NaN,1.0,Democrat,https://www.carper.senate.gov,513 Hart Senate Office Building Washington DC ...,202-224-2441,https://www.carper.senate.gov/contact/,http://www.carper.senate.gov/public/index.cfm/...,SenatorCarper,249787913.0,tomcarper,senatorcarper,UCgLnvbKwu4B3navofj6Qvvw,NaN,C000174,179.0,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,NaN,1.0,Democrat,https://www.casey.senate.gov,393 Russell Senate Office Building Washington ...,202-224-6324,https://www.casey.senate.gov/contact,http://www.casey.senate.gov/rss/feeds/?all,SenBobCasey,171598736.0,SenatorBobCasey,SenatorBobCasey,UCtVssXhx-KuZa-hSvnsnJ0A,NaN,C001070,1828.0,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.


## Match external columns to Wikidata PIDs
At this point, you hould have a dataframe of your external source

In [8]:
# TODO: it would be very cool if there was a gui using something like ipywidgets or a proper web server
def createNamePidMapping(df):
    try:
        with open("colNamePid.json", "r") as file:
            colNamePid = json.load(file)
    except FileNotFoundError:
        colNamePid = {}

    colNamePid["__instructions"] = (
        "# Mapping columns to PIDs and downloading Wikidata data. To skip column, leave 'p' property blank. PIDs should start with capital 'P'. 'pname' is automatically generated"
    )
    for col in df.columns:
        if col in colNamePid:
            print(col, "already in mapping, skipping")
            continue
        first_non_null_index = df[col].first_valid_index()
        if first_non_null_index == None:
            print(col, "has no data, skipping")
            continue
        first_non_null_value = df[col][first_non_null_index]

        colNamePid[col] = {
            "example": str(first_non_null_value),
            "p": "",
            # "pname": None,
        }

    with open("colNamePid.json", "w") as file:
        json.dump(colNamePid, file, indent=2)


createNamePidMapping(politicians)

last_name already in mapping, skipping
first_name already in mapping, skipping
middle_name already in mapping, skipping
suffix already in mapping, skipping
nickname already in mapping, skipping
full_name already in mapping, skipping
birthday already in mapping, skipping
gender already in mapping, skipping
type already in mapping, skipping
state already in mapping, skipping
district already in mapping, skipping
senate_class already in mapping, skipping
party already in mapping, skipping
url already in mapping, skipping
address already in mapping, skipping
phone already in mapping, skipping
contact_form already in mapping, skipping
rss_url already in mapping, skipping
twitter already in mapping, skipping
twitter_id already in mapping, skipping
facebook already in mapping, skipping
youtube already in mapping, skipping
youtube_id already in mapping, skipping
mastodon already in mapping, skipping
bioguide_id already in mapping, skipping
thomas_id already in mapping, skipping
opensecrets_id 

In [9]:
with open("colNamePid.json", "r") as file:
    colNamePid = json.load(file)
colNamePid

{'__instructions': "# Mapping columns to PIDs and downloading Wikidata data. To skip column, leave 'p' property blank. PIDs should start with capital 'P'. 'pname' is automatically generated",
 'last_name': {'example': 'Brown', 'p': 'P734'},
 'first_name': {'example': 'Sherrod', 'p': 'P735'},
 'middle_name': {'example': 'L.', 'p': ''},
 'suffix': {'example': 'Jr.', 'p': 'P8017'},
 'nickname': {'example': 'Bob', 'p': 'P1449'},
 'full_name': {'example': 'Sherrod Brown', 'p': ''},
 'birthday': {'example': '1952-11-09', 'p': 'P569'},
 'gender': {'example': 'M', 'p': 'P21'},
 'type': {'example': 'sen', 'p': 'P39'},
 'state': {'example': 'OH', 'p': ''},
 'district': {'example': '4.0', 'p': 'P39.P768'},
 'senate_class': {'example': '1.0', 'p': ''},
 'party': {'example': 'Democrat', 'p': 'P102'},
 'url': {'example': 'https://www.brown.senate.gov', 'p': 'P856'},
 'address': {'example': '503 Hart Senate Office Building Washington DC 20510',
  'p': 'P937'},
 'phone': {'example': '202-224-2315', 'p

In [10]:
def get_property_name(pid):
    # URL for Wikidata API to get information about the property
    url = (
        f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={pid}&format=json"
    )

    try:
        # Sending a GET request to the Wikidata API
        response = requests.get(url)
        data = response.json()

        # Extracting the English label of the property
        label = data["entities"][pid]["labels"]["en"]["value"]

        return label
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


for key, item in tqdm(colNamePid.items()):
    # Ignore comments
    if key.startswith("__") or len(item["p"]) == 0:
        continue
    pname = get_property_name(item["p"])
    if pname is None:
        print(item["p"] + " is an invalid pid")
        continue
        # TODO: handle this
    item["pname"] = pname

with open("colNamePid.json", "w") as file:
    json.dump(colNamePid, file, indent=2)

 39%|███▉      | 14/36 [00:01<00:02,  9.69it/s]

An error occurred: 'entities'
P39.P768 is an invalid pid


 61%|██████    | 22/36 [00:03<00:02,  5.99it/s]

An error occurred: 'entities'
P2002.P6552 is an invalid pid


 67%|██████▋   | 24/36 [00:03<00:02,  5.74it/s]

An error occurred: 'entities'
P2397.P11245 is an invalid pid


100%|██████████| 36/36 [00:04<00:00,  8.07it/s]
